# BD-NSCA QLoRA Fine-tuning for NPC Dialogue

This notebook fine-tunes Phi-3 Mini for Vietnamese NPC dialogue using QLoRA.

## Requirements
- Google Colab with GPU (T4 or better)
- Training data in JSONL format

In [ ]:
# Install dependencies
!pip install -q transformers==4.40.0 peft==0.10.0 bitsandbytes==0.43.0
!pip install -q accelerate==0.29.0 trl==0.8.0 datasets==2.18.0

In [ ]:
# Upload your training data
from google.colab import files
uploaded = files.upload()  # Upload train.jsonl

In [ ]:
import torch
import json
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import Dataset

print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Configuration
BASE_MODEL = "microsoft/Phi-3-mini-4k-instruct"
DATA_FILE = "train.jsonl"  # Your uploaded file
OUTPUT_DIR = "./npc-phi3-adapter"

# Training params
EPOCHS = 3
BATCH_SIZE = 4
LEARNING_RATE = 2e-4
MAX_SEQ_LENGTH = 1024

# LoRA params
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05

In [ ]:
# Load dataset
def load_jsonl(path):
    samples = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data = json.loads(line)
                samples.append({'text': data['prompt'] + data['completion']})
    return samples

samples = load_jsonl(DATA_FILE)
dataset = Dataset.from_list(samples)
print(f"Loaded {len(dataset)} samples")

In [ ]:
# Setup 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("Model loaded!")

In [ ]:
# Prepare for training
model = prepare_model_for_kbit_training(model)

# LoRA config
peft_config = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=LEARNING_RATE,
    weight_decay=0.001,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="none",
)

In [ ]:
# Initialize trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    tokenizer=tokenizer,
    args=training_args,
    packing=False,
)

# Train!
print("Starting training...")
trainer.train()

In [ ]:
# Save adapter
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Adapter saved to {OUTPUT_DIR}")

In [ ]:
# Test inference
from peft import PeftModel

test_prompt = """[SYSTEM]
Ban la Than giu cong, mot chien binh gia nghiem khac nhung cong bang.

Cot truyen: Nguoi choi den cong lang trong luc lang dang cang thang.

Ngu canh hien tai:
- Trang thai: Dang canh gac
- Tam trang: Nghi ngo
[/SYSTEM]

[USER]
Xin chao, toi muon vao lang.
[/USER]

[ASSISTANT]
"""

inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100, temperature=0.7)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

In [ ]:
# Download adapter
!zip -r npc-phi3-adapter.zip {OUTPUT_DIR}
files.download('npc-phi3-adapter.zip')